# Objective of this Notebook

We will use an SVM classifier to predict if a credit card transaction is fraudulent or not. 

We'll use a dataset from kaggle. 
You can download it at https://www.kaggle.com/mlg-ulb/creditcardfraud

The dataset contains transactions made by credit cards in September 2013 by european cardholders.
This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.



# Table of contents
- [1. Data Exploration](#1-data-exploration)
  * [Reading data](#reading-data1)
  * [Features types](#features-types1)
  * [Missing values](#missing-values2)
  * [Target classes repartition](#target-classes-repartition1)
- [2. Data preprocessing](#2-data-preprocessing2)
  * [Feature scaling](#feature-scaling1)
- [3. Modeling](#3-svm)
  * [Creation of training / validation / test sets](#creation-of-training---validation---test-sets)
  * [Support Vector Classification](#support-vector-classification-params)
    + [The kernel](#the-kernel)
    + [C : strength of regularization](#c---strength-of-regularization)
  * [Test of the best model](#test-of-the-best-model)


## 1. Data Exploration
<a id='1-data-exploration'></a>

### Reading data
<a id='reading-data1'></a>

In [1]:
import pandas as pd
import os
dataset = pd.read_csv('creditcard.csv')
dataset.head()

Time        V1        V2        V3        V4        V5        V6        V7  \
0   0.0 -1.359807 -0.072781  2.536347  1.378155 -0.338321  0.462388  0.239599   
1   0.0  1.191857  0.266151  0.166480  0.448154  0.060018 -0.082361 -0.078803   
2   1.0 -1.358354 -1.340163  1.773209  0.379780 -0.503198  1.800499  0.791461   
3   1.0 -0.966272 -0.185226  1.792993 -0.863291 -0.010309  1.247203  0.237609   
4   2.0 -1.158233  0.877737  1.548718  0.403034 -0.407193  0.095921  0.592941   

         V8        V9  ...       V21       V22       V23       V24       V25  \
0  0.098698  0.363787  ... -0.018307  0.277838 -0.110474  0.066928  0.128539   
1  0.085102 -0.255425  ... -0.225775 -0.638672  0.101288 -0.339846  0.167170   
2  0.247676 -1.514654  ...  0.247998  0.771679  0.909412 -0.689281 -0.327642   
3  0.377436 -1.387024  ... -0.108300  0.005274 -0.190321 -1.175575  0.647376   
4 -0.270533  0.817739  ... -0.009431  0.798278 -0.137458  0.141267 -0.206010   

        V26       V27       V28  Amount  Class  
0 -0.189115  0.133558 -0.021053  149.62      0  
1  0.125895 -0.008983  0.014724    2.69      0  
2 -0.139097 -0.055353 -0.059752  378.66      0  
3 -0.221929  0.062723  0.061458  123.50      0  
4  0.502292  0.219422  0.215153   69.99      0  

[5 rows x 31 columns]

### Features types
<a id='features-types1'></a>

In [2]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
Time      284807 non-null float64
V1        284807 non-null float64
V2        284807 non-null float64
V3        284807 non-null float64
V4        284807 non-null float64
V5        284807 non-null float64
V6        284807 non-null float64
V7        284807 non-null float64
V8        284807 non-null float64
V9        284807 non-null float64
V10       284807 non-null float64
V11       284807 non-null float64
V12       284807 non-null float64
V13       284807 non-null float64
V14       284807 non-null float64
V15       284807 non-null float64
V16       284807 non-null float64
V17       284807 non-null float64
V18       284807 non-null float64
V19       284807 non-null float64
V20       284807 non-null float64
V21       284807 non-null float64
V22       284807 non-null float64
V23       284807 non-null float64
V24       284807 non-null float64
V25       284807 non-null float64
V26  

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, we cannot provide the original features and more background information about the data. Features V1, V2, … V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. 

Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

### Missing values
<a id='missing-values2'></a>

In [3]:
dataset.isnull().any().any()

False

The dataset is pretty convenient, there is no missing values.

### Target classes repartition
<a id='target-classes-repartition1'></a>

In [4]:
negative_class = dataset['Class'].value_counts()[0]
positive_class = dataset['Class'].value_counts()[1]
total =  positive_class + negative_class
print('Percentage of negative class (not fraudulent):',(negative_class / total)*100, "%")
print('Percentage of class (fraudulent):',(positive_class / total)*100, "%")

Percentage of negative class (not fraudulent): 99.82725143693798 %
Percentage of class (fraudulent): 0.1727485630620034 %


The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

Given the class imbalance ratio, it is recommended measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC).                    Confusion matrix accuracy is not meaningful for unbalanced classification.

## 2. Data preprocessing
<a id='2-data-preprocessing2'></a>
### Feature scaling
<a id='feature-scaling1'></a>

Before applying data to SVM, it is important to perform scaling of that data. Main purpose of scaling data before
processing is to avoid attributes in greater numeric ranges. Other purpose is to avoid some types of numerical difficulties
during calculation.  Large attribute values might cause numerical problems.

Two techniques we can use: 

* Feature scaling involves dividing the input values by the range (i.e. the maximum value minus the minimum value) of the input variable, resulting in a new range of just 1. 

* Mean normalization involves subtracting the average value for an input variable from the values for that input variable resulting in a new average value for the input variable of just zero. 

-> To implement both of these techniques, we can adjust our input values as shown in this formula:
$Z = \frac{x - \mu}{\sigma}$ where x is the feature value, $\mu$ is the average of all the values for the feature concerned and $\sigma$ is the standard deviation (or the range value : max-min)





In [5]:
dataset.agg([min, max])

Time        V1         V2         V3         V4          V5  \
min       0.0 -56.40751 -72.715728 -48.325589  -5.683171 -113.743307   
max  172792.0   2.45493  22.057729   9.382558  16.875344   34.801666   

            V6          V7         V8         V9  ...        V21        V22  \
min -26.160506  -43.557242 -73.216718 -13.434066  ... -34.830382 -10.933144   
max  73.301626  120.589494  20.007208  15.594995  ...  27.202839  10.503090   

           V23       V24        V25       V26        V27        V28    Amount  \
min -44.807735 -2.836627 -10.295397 -2.604551 -22.565679 -15.430084      0.00   
max  22.528412  4.584549   7.519589  3.517346  31.612198  33.847808  25691.16   

     Class  
min      0  
max      1  

[2 rows x 31 columns]

Here we can see, huge differences between features intervals. Thus we should perform feature scaling.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

#Features column names without the target 'target' 
features_columns_names = dataset.columns[:-1]
scaled_data = scaler.fit_transform(dataset[features_columns_names])

#Replacing features old values with their scaled values
dataset.loc[:, features_columns_names] = scaled_data

In [7]:
dataset.describe()

Time            V1            V2            V3            V4  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean  -1.050379e-14 -8.157366e-16  3.154853e-17 -4.409878e-15 -6.734811e-16   
std    1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00   
min   -1.996583e+00 -2.879855e+01 -4.403529e+01 -3.187173e+01 -4.013919e+00   
25%   -8.552120e-01 -4.698918e-01 -3.624707e-01 -5.872142e-01 -5.993788e-01   
50%   -2.131453e-01  9.245351e-03  3.965683e-02  1.186124e-01 -1.401724e-02   
75%    9.372174e-01  6.716939e-01  4.867202e-01  6.774569e-01  5.250082e-01   
max    1.642058e+00  1.253351e+00  1.335775e+01  6.187993e+00  1.191874e+01   

                 V5            V6            V7            V8            V9  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean  -2.874435e-16  4.168992e-16 -8.767997e-16 -2.423604e-16  3.078727e-16   
std    1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00   
min   -8.240810e+01 -1.963606e+01 -3.520940e+01 -6.130252e+01 -1.222802e+01   
25%   -5.010686e-01 -5.766822e-01 -4.478860e-01 -1.746805e-01 -5.853631e-01   
50%   -3.936682e-02 -2.058046e-01  3.241723e-02  1.871982e-02 -4.681169e-02   
75%    4.433465e-01  2.991625e-01  4.611107e-01  2.740785e-01  5.435305e-01   
max    2.521413e+01  5.502015e+01  9.747824e+01  1.675153e+01  1.419494e+01   

       ...           V21           V22           V23           V24  \
count  ...  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean   ...  1.685077e-17  1.478472e-15 -6.797197e-16  1.234659e-16   
std    ...  1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00   
min    ... -4.741907e+01 -1.506565e+01 -7.175446e+01 -4.683638e+00   
25%    ... -3.109433e-01 -7.473476e-01 -2.591784e-01 -5.854676e-01   
50%    ... -4.009429e-02  9.345377e-03 -1.792420e-02  6.765678e-02   
75%    ...  2.537392e-01  7.283360e-01  2.364319e-01  7.257153e-01   
max    ...  3.703471e+01  1.447304e+01  3.607668e+01  7.569684e+00   

                V25           V26           V27           V28        Amount  \
count  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05  2.848070e+05   
mean  -7.659279e-16  3.247603e-16 -2.953495e-18  5.401572e-17  3.202236e-16   
std    1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00   
min   -1.975033e+01 -5.401098e+00 -5.590660e+01 -4.674612e+01 -3.532294e-01   
25%   -6.084001e-01 -6.780717e-01 -1.755053e-01 -1.604440e-01 -3.308401e-01   
50%    3.183240e-02 -1.081217e-01  3.325174e-03  3.406368e-02 -2.652715e-01   
75%    6.728006e-01  4.996663e-01  2.255648e-01  2.371526e-01 -4.471707e-02   
max    1.442532e+01  7.293975e+00  7.831940e+01  1.025434e+02  1.023622e+02   

               Class  
count  284807.000000  
mean        0.001727  
std         0.041527  
min         0.000000  
25%         0.000000  
50%         0.000000  
75%         0.000000  
max         1.000000  

[8 rows x 31 columns]

Here the features have been transformed with feature scaling and normalization so that they have the properties of a standard normal distribution i.e with mean = 0 and standard deviation = 1.
This will speed up calcultations of algorithms using Gradient Descent and measures of distance.

## 3. Modeling
<a id='3-svm'></a>
### Creation of training / validation / test sets
<a id='creation-of-training---validation---test-sets'></a>
We will create 3 datasets : training set (60% of all the dataset), validation set (20%), test set (20%)

In [8]:
from sklearn.model_selection import train_test_split 

target = dataset['Class']
features = dataset.drop(columns=['Class'])

#First we create a training (60% of the entire dataset) and test set (40% of the entire dataset)
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.40, random_state=42, shuffle=True, stratify=target)

#Afterwards, We split the test set in 2 equal parts : the validation set and the new and smaller test set.
X_test, X_validation, y_test, y_validation = train_test_split(X_test, y_test, test_size=0.50, random_state=42, shuffle=True, stratify=y_test)


print('X_train shape:',X_train.shape, ' & y_train shape:',y_train.shape)
print('X_validation shape:',X_validation.shape,' & y_validation shape:',y_validation.shape)
print('X_test shape:',X_test.shape,' & y_test shape:',y_test.shape)

X_train shape: (170884, 30)  & y_train shape: (170884,)
X_validation shape: (56962, 30)  & y_validation shape: (56962,)
X_test shape: (56961, 30)  & y_test shape: (56961,)


### Support Vector Classification parameters
<a id='support-vector-classification-params'></a>

We will train different classifiers using different combination value of parameters.
The parameters are :
#### The kernel
<a id='the-kernel'></a>
It is a simlilarity function used to learn complex or less complex decision boundaries.
We will use 2 of the most used kernels, the linear and the gaussian kernel.
The liner kernel is the most simple, it separates data with a line which is reasonable when data is linearly separable.
The gaussian kernel is a more complex kernel, intended to separate non linearly separable data.

<img src="lin.png" />



#### C : strength of regularization
<a id='c---strength-of-regularization'></a>
C is a regularization parameter which must be strictly positive.

For this parameter C, we should take into consideration two things :
* A small value for C, means a small penalization which will not encourage the model to classify each example well. This can cause a high bias and a low variance which means underfitting.
* A large value for C, means a higher penalization which will encourage the model to classify each example well. This can cause a lower bias but a higher variance which means overfitting.


The parameter C will have several numerical values.

In [9]:
from sklearn.svm import SVC
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
    
kernels = ['linear', 'rbf' ]
C = [0.1, 0.3, 1, 3, 10]
results = []
for kernel in kernels:
    for c in C:
        svc = SVC(C = c, kernel = kernel)
        svc = svc.fit(X_train, y_train)
        predictions = svc.predict(X_validation)
        precision, recall, _ = precision_recall_curve(y_validation.values, predictions)
        precision_recal_auc = auc(recall, precision)
        results.append([c,kernel,precision_recal_auc])

As explained previously, given the class imbalance ratio, it is recommended measuring the accuracy using the Area Under the Precision-Recall Curve (AUPRC). This is what we used here when doing the ' auc(recall,precision)' step which is computing the area under the precision-recall curve (auprc).

We can now : 
* figure out the best parameters giving the best auprc
* build the best model with the best parameters
* test the generalization of the best model on the test set.

### Test of the best model
<a id='test-of-the-best-model'></a>


In [10]:
def getBestParameters(results):
    best_c = results[0][0]
    best_kernel = results[0][1]
    highest_auprc = results[0][2]
    for i in range(1,len(results)):
        current_auprc = results[i][2]
        if (current_auprc > highest_auprc):
            highest_auprc = current_auprc
            best_c = results[i][0]
            best_kernel = results[i][1]
    return [best_c,best_kernel, highest_auprc]

best_params = getBestParameters(results)
best_c = best_params[0]
best_kernel = best_params[1]
highest_auprc = best_params[2]


print('Best parameters:',best_params)

svc = SVC(C = best_c, kernel = best_kernel)
svc = svc.fit(X_train, y_train)
test_predictions = svc.predict(X_test)
precision, recall, _ = precision_recall_curve(y_test.values, test_predictions)
precision_recal_auc = auc(recall, precision)

print("Test set, area under precision-recall value:",precision_recal_auc)
        

Best parameters: [3, 'rbf', 0.8285812728599148]
Test set, area under precision-recall value: 0.8098635014125732
